### TP3 - 2

2. O seguinte sistema dinâmico denota 4 inversores ($\,A, B, C, D\,$) que lêm um bit num canal input e escrevem num canal output uma transformação desse bit.


<img src="inversores.png" alt="autómato"/>


   1. Cada inversor tem um bit $s$ de estado, inicializado  com um valor aleatório.
   2. Cada inversor é regido pelas seguintes transformações 
  
     $$ \mathbf{invert}\mathtt(in,out) $$
     $$x \gets \mathsf{read}(\mathtt{in})$$
     $$s \gets \neg x\;\;\|\;\; s\gets s\oplus x$$             
     $$\mathsf{write}(\mathtt{out},s)$$
     
        iii. A escolha neste comando é sempre determinística; isto é, em cada inversor a escolha do comando a executar é sempre a mesma. Porém qual é essa escolha é determinada aleatoriamente na inicializarão do sistema.
        
        iii. O estado do sistema é um duplo definido pelos 4 bits $s$, e é inicializado com um vetor aleatório em $\{0,1\}^4\;$.
        
        iv. O sistema termina em ERRO quando o estado do sistema for $\,(0,0,0,0)\,$.


1. Construa um SFOTS que descreva este sistema e implemente este sistema, numa abordagem BMC (“bouded model checker”) num traço com $\,n\,$ estados.
2. Verifique se o sistema é seguro usando BMC, k-indução ou model checking com interpolantes.

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import INT
import itertools 
from random import randint

### Função genState bla bla bla

In [2]:
# vars = ['pc', 's']
def genState(vars,s,i,n):
    state = {}
    state['pc'] = Symbol('pc'+'_'+s+str(i), BVType(n))
    for j in range(1, 5):
        for v in vars[1:]:
            if v not in state:
                state[v] = {}
            
            state[v][j] = Symbol(v+str(j)+'_'+s+str(i), BVType(n))
            
    #print(state)
    
            
    return state

#### Função init(state):

    state - Dicionário de variáveis de estado


   A função `init` tem como objetivo devolver um predicado do Solver que testa se é um possível estado inicial do programa, através do `state`, um dicionário de variáveis.

In [3]:
def init(state, n):
    
    random_state = (BV(randint(0,1), n), BV(randint(0,1), n), BV(randint(0,1), n), BV(randint(0,1), n)) 
    

    s_list = []
    for v in state['s']:
        s_list.append( Equals(state['s'][v], random_state[v-1]) )
    
    return And(
        Equals(state['pc'], BV(1, n)),
        And(s_list)
    )

#### Função trans(curr, prox):

    curr - Estado das variáveis no momento atual
    prox - Estado das variáveis no momento da próxima iteração
    
   A função `trans` tem como objetivo devolver um predicado do Solver, através dos três estados disponíveis, que teste se é possível transitar entre os estados possíveis.

In [4]:
def error(state, n):
    return Or(
        Equals( BVAdd(
                    BVAdd(state['s'][1], state['s'][2]),
                    BVAdd(state['s'][3], state['s'][4])
               ), 
               BV(0, n)
        )
    )


def trans(curr, prox, n):
    zero_state = Equals( 
                BVAdd(
                    BVAdd(curr['s'][1], curr['s'][2]),
                    BVAdd(curr['s'][3], curr['s'][4])
            ),  BV(0, n)
    )
    
    # prox_x = prox_s
    # prox_s = Not(curr[x])  V Xor(curr[s], curr[x])  
    t1_2 = And(
        Equals(curr['pc'], BV(1, n)),
            
        Or(
            And(Not(zero_state), Equals(prox['pc'], BV(2, n))),
            And(zero_state, Equals(prox['pc'], BV(5, n)))
        ),
        
#       0 0 0 0 1
        
#       1 1 1 1 0
#     - 1 1 1 1 0
#      ------------
#       0 0 0 0 0
        
#       0 0 0 0 0

#       1 1 1 1 1
#     - 1 1 1 1 0
#      ------------
#       0 0 0 0 1
        
        
        
        # o x é o s do inversor anterior
        # prox_s = Not(curr[x])  V Xor(curr[s], curr[x])
        Equals(prox['s'][1], BVOr( BVSub(BVNot(curr['s'][4]), BV(30, n)), BVXor(curr['s'][1], curr['s'][4]) )), 
        
        And([ Equals(prox['s'][i], curr['s'][i]) for i in range(2,5)])
        
    )  
    
    t2_3 = And(
        Equals(curr['pc'], BV(2, n)),
            
        Or(
            And(Not(zero_state), Equals(prox['pc'], BV(3, n))),
            And(zero_state, Equals(prox['pc'], BV(5, n)))
        ),
        
        Equals(prox['s'][2], BVOr( BVSub(BVNot(curr['s'][1]), BV(30, n)), BVXor(curr['s'][2], curr['s'][1]) )),
        
        Equals(prox['s'][1], curr['s'][1]),
        And([Equals(prox['s'][i], curr['s'][i]) for i in range(3,5)]) 
    )
    
    t3_4 = And(
        Equals(curr['pc'], BV(3, n)),
            
        Or(
            And(Not(zero_state), Equals(prox['pc'], BV(4, n))),
            And(zero_state, Equals(prox['pc'], BV(5, n)))
        ),
        
        Equals(prox['s'][3], BVOr( BVSub(BVNot(curr['s'][2]), BV(30, n)), BVXor(curr['s'][3], curr['s'][2]) )),
        
        And([Equals(prox['s'][i], curr['s'][i]) for i in range(1,3)]),
        Equals(prox['s'][4], curr['s'][4])
    )
    
    t4_1 = And(
        Equals(curr['pc'], BV(4, n)),
            
        Or(
            And(Not(zero_state), Equals(prox['pc'], BV(1, n))),
            And(zero_state, Equals(prox['pc'], BV(5, n)))
        ),
        
        Equals(prox['s'][4], BVOr( BVSub(BVNot(curr['s'][3]), BV(30, n)), BVXor(curr['s'][4], curr['s'][3]) )),
        
        And([Equals(prox['s'][i], curr['s'][i]) for i in range(1,4)])
    )
    
    
    error = And(
        Equals(curr['pc'], BV(5, n)),
        Equals(prox['pc'], curr['pc']),
        zero_state,
        And( [Equals(prox['s'][i], curr['s'][i]) for i in range(1,5)])
    )
    
    return Or(t1_2, t2_3, t3_4, t4_1, error)
    

#### Função gera_traco(declare, init, trans, k, n, a, b)

    declare - Variáveis de estado
    init - Condições para o estado inicial
    trans - Função transição
    k - Valor do traço


A função `gera_traco` tem como objetivo imprimir o valor das variáveis à medida que vão percorrendo os estados, através das variáveis do estado e de um predicado que testa se um estado é inicial.

In [5]:
def gera_traco(vars, init, trans, error, n, k):
    with Solver(name="z3") as s:
    
        X = [genState(vars,'X',i, n) for i in range(k+1)]
        #print(X)
        I = init(X[0], n)
        print(I)
        Tks = [ trans(X[i],X[i+1],n) for i in range(k) ]
        #print(Tks)
    
        if s.solve([I,And(Tks)]):
            for i in range(k):
                print("Passo:",i)
                for v in X[i]:
                    if v == 'pc':
                        print("Estado: ",s.get_value(X[i][v]))
                    else:
                        for k in range(1,5):
                            print("          ",v,'=',s.get_value(X[i][v][k]))
                print("----------------")
        else:
            print("unsat")
        
        
gera_traco(['pc', 's'], init, trans, error, 5, 20)

((pc_X0 = 1_5) & ((s1_X0 = 0_5) & (s2_X0 = 0_5) & (s3_X0 = 0_5) & (s4_X0 = 1_5)))
Passo: 0
Estado:  1_5
           s = 0_5
           s = 0_5
           s = 0_5
           s = 1_5
----------------
Passo: 1
Estado:  2_5
           s = 1_5
           s = 0_5
           s = 0_5
           s = 1_5
----------------
Passo: 2
Estado:  3_5
           s = 1_5
           s = 1_5
           s = 0_5
           s = 1_5
----------------
Passo: 3
Estado:  4_5
           s = 1_5
           s = 1_5
           s = 1_5
           s = 1_5
----------------
Passo: 4
Estado:  1_5
           s = 1_5
           s = 1_5
           s = 1_5
           s = 0_5
----------------
Passo: 5
Estado:  2_5
           s = 1_5
           s = 1_5
           s = 1_5
           s = 0_5
----------------
Passo: 6
Estado:  3_5
           s = 1_5
           s = 0_5
           s = 1_5
           s = 0_5
----------------
Passo: 7
Estado:  4_5
           s = 1_5
           s = 0_5
           s = 1_5
           s = 0_5
---------------

#### ii) Verificação com interpolantes

In [24]:
def invert(trans, n_bits):
    return (lambda c, p: trans(p,c, n_bits))

def baseName(s):
    return ''.join(list(itertools.takewhile(lambda x: x!='_', s)))

def rename(form,state):
    vs = list(get_free_variables(form))
    
    pairs = []
    for x in range(1,5):
        if str(vs[1])[0:2] == 'pc':
            pairs.insert(0, (vs[0], state['pc']) )
        else:
            pairs.append( (vs[x], state['s'][x]) )
        
    return form.substitute(dict(pairs))


def same(state1,state2):
    return And( Equals(state1['pc'], state2['pc']), And([Equals(state1['s'][x], state2['s'][x]) for x in state1['s']]))

In [25]:
def model_checking(vars,init,trans,error,n_bits,N,M):
    with Solver(name="z3") as s:

        # Criar todos os estados que poderão vir a ser necessários.
        X = [genState(vars,'X',i, n_bits) for i in range(N+1)]
       # print(X)
        Y = [genState(vars,'Y',i, n_bits) for i in range(M+1)]
        
        # Estabelecer a ordem pela qual os pares (n,m) vão surgir. Por exemplo:
        order = sorted([(a,b) for a in range(1,N+1) for b in range(1,M+1)],key=lambda tup:tup[0]+tup[1]) 
        
        for (n,m) in order:
            Tn = And([trans(X[i], X[i+1], n_bits) for i in range(n)])
            I = init(X[0], n_bits) 
            Rn = And(I, Tn)
            
            Bm = And([invert(trans, n_bits)(Y[i], Y[i+1]) for i in range(m)])
            E = error(Y[0], n_bits)
            Um = And(E, Bm)
            
            Vnm = And(Rn, same(X[n], Y[m]), Um)
                
            if s.solve([Vnm]):
                print("Unsafe")
                return
            else:                        # Vnm é instatisfazível
                C = binary_interpolant(And(Rn, same(X[n], Y[m])), Um)
                if C is None:
                    print("Interpolant None")
                    break
                C0 = rename(C, X[0])
                C1 = rename(C, X[1])
                T = trans(X[0], X[1], n_bits)
                if not s.solve([C0, T, Not(C1)]):   # C é invariante de T
                    print("Safe")
                    return
                else:
                    ### tenta gerar o majorante S
                    S = rename(C, X[n])
                    while True:
                        A = And(S, trans(X[n], Y[m], n_bits))
                        if s.solve([A,Um]):
                            print("Não é possível encontrar um majorante")
                            break
                        else:
                            Cnew = binary_interpolant(A, Um)
                            Cn = rename(Cnew, X[n])
                            if s.solve([Cn, Not(S)]):   # Se Cn -> S não é tautologia
                                S = Or(S, Cn)
                            else:             # S foi encontrado
                                print("Safe")
                                return
                            
            print("unknown")                   


model_checking(['pc', 's'], init, trans, error, 5, 50, 50)

Safe
